In [255]:
import pandas as pd
from sodapy import Socrata

# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("data.cdc.gov", None)

# Example authenticated client (needed for non-public datasets):
# client = Socrata(data.cdc.gov,
#                  MyAppToken,
#                  username="user@example.com",
#                  password="AFakePassword")

# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("n8mc-b4w4",limit = 9000000)

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)

In [256]:
print(len(results_df))
# results_df.head(500)

9000000


In [257]:
# I guess when discussing whether or not to drop NaN's or Missing's, we should consider our ultimate goal
# for the time being, I want to predict hosp_yn
# Thus I will drop any unknown and missings from that col, as well as not consider icu_yn or death_yn

# first I will drop icu and death info
# i will drop county and state info - but keep state_code
results_df.drop(['res_state','res_county','process','ethnicity'], axis = 1, inplace = True)


In [258]:
results_df.head(500)

,case_month,state_fips_code,county_fips_code,age_group,sex,race,case_onset_interval,exposure_yn,current_status,symptom_status,hosp_yn,icu_yn,death_yn,case_positive_specimen,underlying_conditions_yn
0,2020-10,19,19193,0 - 17 years,Female,White,0,Unknown,Laboratory-confirmed case,Symptomatic,No,No,No,NaN,NaN
1,2020-11,48,48349,18 to 49 years,Female,White,-1,Missing,Probable Case,Symptomatic,No,Missing,No,NaN,NaN
2,2021-08,46,46135,18 to 49 years,Female,White,NaN,Missing,Laboratory-confirmed case,Missing,Missing,Missing,Missing,NaN,NaN
3,2021-12,27,27109,65+ years,Female,White,NaN,Missing,Laboratory-confirmed case,Missing,Missing,Missing,No,0,NaN
4,2021-09,26,26129,0 - 17 years,Male,White,0,Missing,Laboratory-confirmed case,Symptomatic,No,Missing,No,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,2021-08,35,35015,18 to 49 years,Male,Missing,NaN,Missing,Laboratory-confirmed case,Missing,Missing,Missing,Missing,NaN,NaN
496,2022-01,04,04005,18 to 49 years,Female,White,0,Missing,Laboratory-confirmed case,Symptomatic,No,Missing,No,NaN,NaN
497,2022-01,51,51075,18 to 49 years,Female,White,NaN,Missing,Laboratory-confirmed case,Missing,Missing,Missing,Missing,NaN,NaN
498,2021-08,21,21027,50 to 64 years,Female,White,NaN,Missing,Laboratory-confirmed case,Missing,No,Missing,No,0,NaN


In [259]:
# dropping all NaN's just to see
results_df.dropna(axis = 0, inplace = True)

In [260]:
print(len(results_df))
results_df.head()

276185


,case_month,state_fips_code,county_fips_code,age_group,sex,race,case_onset_interval,exposure_yn,current_status,symptom_status,hosp_yn,icu_yn,death_yn,case_positive_specimen,underlying_conditions_yn
85,2020-10,49,49035,50 to 64 years,Male,White,0,Yes,Laboratory-confirmed case,Symptomatic,No,No,No,0,Yes
111,2021-09,34,34037,18 to 49 years,Female,White,0,Missing,Laboratory-confirmed case,Symptomatic,No,Missing,No,0,Yes
124,2021-01,49,49057,18 to 49 years,Male,White,0,Yes,Laboratory-confirmed case,Symptomatic,No,No,No,0,Yes
139,2022-01,47,47035,18 to 49 years,Female,White,0,Missing,Laboratory-confirmed case,Symptomatic,No,No,No,0,Yes
217,2021-07,22,22105,0 - 17 years,Male,Black,0,Missing,Probable Case,Symptomatic,No,Unknown,Missing,0,Yes


In [277]:
# so it seems there are some NA's and Missing's and Unknown's to remove as well
results_df.drop(results_df[(results_df['sex'] == "NA") | (results_df['sex'] == "Unknown")].index, inplace = True)
print(len(results_df))

123111


In [262]:
# remove missing exposure data - keeping unknown
# results_df.drop(results_df[results_df['exposure_yn'] == "Missing"].index, inplace = True)
# print(len(results_df))

In [263]:
# cleaning race
results_df.drop(results_df[(results_df['race'] == "NA") | (results_df['race'] == "Missing") | (results_df['race'] == "Unknown")].index, inplace = True)
print(len(results_df))

212832


In [264]:
# cleaning age
results_df.drop(results_df[(results_df['age_group'] == "Missing") | (results_df['age_group'] == "Unknown")].index, inplace = True)
print(len(results_df))

212832


In [265]:
# results_df.drop(results_df[results_df['ethnicity'] == "NA"].index, inplace = True)
# ethinicity may be removed - it is redundant given race ???

In [266]:
# cleaning hosp
# results_df.drop(results_df[(results_df['hosp_yn'] == "Unknown") | (results_df['hosp_yn'] == "Missing")].index, inplace = True)
# print(len(results_df))

In [267]:
# what to do with process??? --- For now I drop process, it took so much data away when removing unknowns
# i am going to get rid of unknown and missing
#results_df.drop(results_df[(results_df['process'] == "Missing") | (results_df['process'] == "Unknown")].index, inplace = True)
#print(len(results_df))

In [268]:
# exposure only has a yes or unknown/missing - unsure if fit for use (???)
# i would argue to remove missing - the col can be seen as known to have been exposed
results_df.drop(results_df[results_df['exposure_yn'] == "Missing"].index, inplace = True)
print(len(results_df))

123519


In [269]:
results_df.drop(results_df[results_df['underlying_conditions_yn'] == "blank"].index, inplace = True)
print(len(results_df))

123519


In [270]:
# get rid of missing symptom_status
results_df.drop(results_df[results_df['symptom_status'] == "Missing"].index, inplace = True)
print(len(results_df))

123519


In [271]:
# NOTE: not getting rid of missing death/icu info, there may exist the case where hosp/icu is known for a row and death is 
# missing, so I will let Brian and Jowaki filter that as needed on thier end

In [278]:
print(len(results_df))
results_df.head(20)

123111


,case_month,state_fips_code,county_fips_code,age_group,sex,race,case_onset_interval,exposure_yn,current_status,symptom_status,hosp_yn,icu_yn,death_yn,case_positive_specimen,underlying_conditions_yn
85,2020-10,49,49035,2,1,5,0,1,1,1,0,0,0,0,1
124,2021-01,49,49057,1,1,5,0,1,1,1,0,0,0,0,1
283,2021-02,22,22009,2,1,5,0,1,0,1,0,-1,-2,0,1
303,2020-11,42,42025,1,1,5,0,1,1,1,0,0,-3,0,1
476,2020-07,42,42003,1,1,2,0,1,1,1,0,0,0,1,1
627,2021-02,22,22075,1,0,5,0,1,0,1,0,-1,-2,0,1
666,2021-02,42,42127,3,1,5,0,1,1,1,-1,0,-3,0,1
705,2020-12,21,21163,2,0,5,0,1,1,1,0,-2,0,1,1
722,2020-04,18,18005,1,1,5,0,1,1,1,0,-2,0,1,1
789,2020-11,05,05125,1,0,5,0,1,1,1,0,-1,0,0,1


In [279]:
# start on the discretizations -

# sex - can be [female, male, other]
results_df['sex'].replace(['Female','Male','Other'], [0,1,2], inplace = True)

# age group [0-17,18-49,50-64,65+] to [0,1,2,3]
results_df['age_group'].replace(['0 - 17 years','18 to 49 years','50 to 64 years',
                                '65+ years'],[0,1,2,3], inplace = True)

# race - [American Indian/Alaska Native; Asian; Black; Multiple/Other; Native Hawaiian/Other Pacific Islander; White]
results_df['race'].replace(['American Indian/Alaska Native', 'Asian', 'Black',
                            'Multiple/Other','Native Hawaiian/Other Pacific Islander',
                            'White'], [0,1,2,3,4,5], inplace = True )

# process - Clinical evaluation; Routine surveillance; Contact tracing of case patient; Multiple; Other;
# results_df['process'].replace(['Clinical evaluation', 'Routine surveillance', 'Contact tracing of case patient',
#                                'Multiple', 'Laboratory reported','Other'], [0,1,2,3,4,5], inplace = True)
                               
# exposure - [yes,Unknown]
results_df['exposure_yn'].replace(['Unknown','Yes','Missing'], [0,1,2], inplace = True)
                               
# current status - [Laboratory-confirmed case, Probable case]
results_df['current_status'].replace(['Laboratory-confirmed case', 'Probable Case'], [1,0], inplace = True)

# symptom status - [Asymptomatic, Symptomatic, Unknown]
results_df['symptom_status'].replace(['Asymptomatic','Symptomatic','Unknown'], [0,1,2], inplace = True)

# hosp_yn - yes or no
results_df['hosp_yn'].replace(['Yes','No','Unknown','Missing'], [1,0,-1,-2], inplace = True)

# underlying conditions - yes or no
results_df['underlying_conditions_yn'].replace(['Yes','No'], [1,0], inplace = True)

# filter icu  - 
results_df['icu_yn'].replace(['Yes','No','Unknown','Missing'], [1,0,-1,-2], inplace = True)

# filter death - 
results_df['death_yn'].replace(['Yes','No','Unknown','Missing','NA'], [1,0,-1,-2,-3], inplace = True)

In [274]:
results_df.head(20)

,case_month,state_fips_code,county_fips_code,age_group,sex,race,case_onset_interval,exposure_yn,current_status,symptom_status,hosp_yn,icu_yn,death_yn,case_positive_specimen,underlying_conditions_yn
85,2020-10,49,49035,2,1,5,0,1,1,1,0,0,0,0,1
124,2021-01,49,49057,1,1,5,0,1,1,1,0,0,0,0,1
283,2021-02,22,22009,2,1,5,0,1,0,1,0,-1,-2,0,1
303,2020-11,42,42025,1,1,5,0,1,1,1,0,0,-3,0,1
476,2020-07,42,42003,1,1,2,0,1,1,1,0,0,0,1,1
627,2021-02,22,22075,1,0,5,0,1,0,1,0,-1,-2,0,1
666,2021-02,42,42127,3,1,5,0,1,1,1,-1,0,-3,0,1
705,2020-12,21,21163,2,0,5,0,1,1,1,0,-2,0,1,1
722,2020-04,18,18005,1,1,5,0,1,1,1,0,-2,0,1,1
789,2020-11,05,05125,1,0,5,0,1,1,1,0,-1,0,0,1


In [281]:
results_df.to_csv('prepro_data.csv', encoding='utf-8', index=False)